In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/full-filled-brain-stroke-dataset/full_data.csv
/kaggle/input/full-filled-brain-stroke-dataset/full_filled_stroke_data (1).csv


In [2]:
from __future__ import absolute_import,division,print_function,unicode_literals

import tensorflow as tf

In [3]:
features=['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status','stroke']

In [4]:
train=pd.read_csv('/kaggle/input/full-filled-brain-stroke-dataset/full_data.csv',names=features,header=0)
test=pd.read_csv('/kaggle/input/full-filled-brain-stroke-dataset/full_filled_stroke_data (1).csv',names=features,header=0)

In [5]:
train['gender']=train['gender'].replace(['Male','Female'],[1,0])
train['ever_married']=train['ever_married'].replace(['Yes','No'],[1,0])
train['work_type']=train['work_type'].replace(['Private','Self-employed','Govt_job','children'],[0,1,2,3])
train['Residence_type']=train['Residence_type'].replace(['Urban','Rural'],[1,0])
train['smoking_status']=train['smoking_status'].replace(['formerly smoked','never smoked','smokes','Unknown'],[0,1,2,3])
test['gender']=test['gender'].replace(['Male','Female'],[1,0])
test['ever_married']=test['ever_married'].replace(['Yes','No'],[1,0])
test['work_type']=test['work_type'].replace(['Private','Self-employed','Govt_job','children'],[0,1,2,3])
test['Residence_type']=test['Residence_type'].replace(['Urban','Rural'],[1,0])
test['smoking_status']=test['smoking_status'].replace(['formerly smoked','never smoked','smokes','Unknown'],[0,1,2,3])

In [6]:
train_y=train.pop('stroke')
test_y=test.pop('stroke')

In [7]:
def input_fn(features,labels,training=True,batch_size=256):
    dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels))
    if training:
        dataset=dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [8]:
my_feature_column=[]
for key in train.keys():
    my_feature_column.append(tf.feature_column.numeric_column(key=key))

In [9]:
classifier=tf.estimator.DNNClassifier(feature_columns=my_feature_column,hidden_units=[30,10],n_classes=2)

In [10]:
classifier.train(input_fn=lambda: input_fn(train,train_y,training=True),steps=5000)

2022-08-24 13:40:58.437519: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
result=classifier.evaluate(input_fn=lambda: input_fn(test,test_y,training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**result))


Test set accuracy: 0.801

